In [ ]:
# default_exp circuit

# Circuit

> Representation of a quantum circuit.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from collections.abc import MutableSequence
from functools import cached_property

A circuit is a list of ticks. A tick consists of gates associated to qubits. We can have one-qubit gates which are applied to single gate locations and two-qubit gates which are applied to an ordered (control,target)-qubit pair. For simplicity, we implement this as a list of dictionaries. Each tick is represented by a dictionary (a *tick*tionary if you may) which holds the `gate_symbol` as key and the set of `qubits` we want to apply these gates to as values. Qubits can be either *integers* for single-qubit locations or *tuples* for two-qubit locations.

In [ ]:
#export
class Circuit(MutableSequence):
    """Representation of a quantum circuit"""
    
    def __init__(self, ticks=None, failures=None):
        self._ticks = ticks if ticks else []
        
    def __getitem__(self, tick_index):
        return self._ticks[tick_index]
    
    def __setitem__(self, tick_index, tick):
        self._ticks[tick_index] = tick
        
    def __delitem__(self, tick_index):
        del self._ticks[tick_index]
        
    def __len__(self):
        return len(self._ticks)
    
    def insert(self, tick_index, tick):
        self._ticks.insert(tick_index, tick)
    
    def __str__(self):
        str_list = []
        for i, tick in enumerate(self._ticks):
            str_list.append(f"{i}: {str(tick)}")
        return "\n".join(str_list)
    
    def __repr__(self):
        return self.__str__()
    
    @property
    def _qubits(self):  
        """Set of qubits used in circuit"""
        return set(unpack(self._ticks))
    
    @cached_property
    def n_qubits(self):
        """Number of qubits used in circuit"""
        return len(self._qubits)
    
    @property
    def n_ticks(self):
        """Number of ticks"""
        return len(self._ticks)

As `Circuit` is a `MutableSequence` we get all the Pythonic list functionality for free. Let's check several ways we can create and add to a circuit.

In [ ]:
circuit = Circuit([{'H': {1,2,3}}])
assert circuit._ticks == [{'H': {1,2,3}}]

In [ ]:
circuit.append({'X': {0,1}, 'CNOT':{(2,3)}})
assert circuit._ticks == [{'H': {1,2,3}}, {'X': {0,1}, 'CNOT':{(2,3)}}]

In [ ]:
circuit += [{'CNOT':{(2,3),(0,1)}}]
assert circuit._ticks == [{'H': {1,2,3}}, {'X': {0,1}, 'CNOT':{(2,3)}}, {'CNOT': {(2,3),(0,1)}}]

In [ ]:
circuit[0] = {'Y': {0,3}}
assert circuit._ticks == [{'Y': {0,3}}, {'X': {0,1}, 'CNOT':{(2,3)}}, {'CNOT': {(2,3),(0,1)}}]

Of course we can also print our `Circuit`. We get a list of ticks and the corresponding ticks which are sequentially applied to our state during simulation.

In [ ]:
print(circuit)

0: {'Y': {0, 3}}
1: {'X': {0, 1}, 'CNOT': {(2, 3)}}
2: {'CNOT': {(2, 3), (0, 1)}}


Furthermore, our `Circuit` class gives us some useful information about the circuit, as

In [ ]:
show_doc(Circuit.n_qubits)
show_doc(Circuit.n_ticks)

<h4 id="Circuit.n_qubits" class="doc_header"><code>Circuit.n_qubits</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Number of qubits used in circuit

<h4 id="Circuit.n_ticks" class="doc_header"><code>Circuit.n_ticks</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Number of ticks

# Circuit helper functions
Sometimes it is useful to get a list of all the qubits which are "touched" in a circuit. From this we can for example infer the total number of qubits in a circuit without ever specifying them directly. Now, we define a `Circuit` as a list of ticks where each tick is represented by a dictionary and the values of these dictionaries contain sets of qubits. These sets can either contain integers or tuples of two integers of qubit indices. Thus, we need a way to unpack (or flatten) all these tuples first before we can merge the indices together in a list. `unpack` accomplishes this recursively.

In [ ]:
#export
def unpack(seq):
    """Generator for all qubits defined in a (sub)circuit"""
    
    if isinstance(seq, (tuple,set,list)):
        yield from (x for y in seq for x in unpack(y))
    elif isinstance(seq, dict):
        yield from (x for v in seq.values() for y in v for x in unpack(y))
    else:
        yield seq

Let's see how it works. We give it a list of dictionaries as a representation for some circuit.

In [ ]:
circuit = [{'H': {1, 2, 3}}, {'X': {0, 1}, 'CNOT': {(2, 3), (1,5)}}]
print(list(unpack(circuit)))

[1, 2, 3, 0, 1, 2, 3, 1, 5]


As we can see, the output does, exactly as we expected, contains a list of all the qubits in the order we defined them in the circuit where tuples are correctly unpacked as well.

Another useful method which is of particular interest for the creation of multi-parameter noise models is `partition`. This function allows us to scan a `Circuit` for certain gates and return a list of `(tick_index,qubits)`-tuples which represent a certain `partition` of a circuit. For example we could pass the gate symbols for all two-qubit gates to `partition` to retrieve a list of all circuit locations where two-qubit gates are present. This list can be subsequently used for example to randomly select faulty locations with some probability.

In [ ]:
#export
def partition(circuit,gates):
    """Find all (tick,qubit) circuit locations for list of `gates`"""
    
    return [(t_idx,q) for (t_idx,tick) in enumerate(circuit) 
            for g,qs in tick.items() for q in qs if g.upper() in gates]

In [ ]:
print(partition(circuit, ["X"]))
print(partition(circuit, ["X","H"]))
print(partition(circuit, ["CNOT"]))

[(1, 0), (1, 1)]
[(0, 1), (0, 2), (0, 3), (1, 0), (1, 1)]
[(1, (2, 3)), (1, (1, 5))]


If we want to use circuits as keys in dictionaries we first need to create a hash, i.e. a unique ID for a particular circuit. As a circuit is in general a mutable object we cannot hash it itself. The external function `make_hash` generates a hash for a circuit.

In [ ]:
#hide
#export
def freeze(obj):
    if isinstance(obj, dict): # dicts to frozensets
        return frozenset({k:freeze(v) for k,v in obj.items()}.items())
    if isinstance(obj, (set,tuple,list)):
        return tuple([freeze(v) for v in obj]) # lists sets and tuples to tuples
    return obj # hashable object

In [ ]:
#export
def make_hash(circuit):
    return hash(freeze(circuit))

In [ ]:
circuit = [{'H': {1, 2, 3}}, {'X': {0, 1}, 'CNOT': {(2, 3), (1,5)}}]
circuit2 = [{'H': {1, 2, 3}}, {'X': {0, 1}}]

print(make_hash(circuit))
print(make_hash(circuit2))

-2227223373693009670
2112759809525957455
